# Analyze runtimes

The referee asked about the total number of samples produced by the different pipelines.

In [175]:
import os
import psutil
p = psutil.Process()
p.cpu_affinity([0])
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import matplotlib.pyplot as plt
import jax.numpy as jnp
import jax
jax.config.update("jax_disable_jit", True)
import arviz

import h5py
import json
from scipy.special import logsumexp

In [176]:
gwosc_path = "/home/thibeau.wouters/gw-datasets/GW190425/posterior_samples.h5"
jim_root_path = "/home/thibeau.wouters/TurboPE-BNS/real_events/"
bilby_root_path = "/home/thibeau.wouters/jim_pbilby_samples/older_bilby_version/"
paths_dict = {"GW170817_TaylorF2": {"jim": jim_root_path + "GW170817_TaylorF2/outdir/results_production.npz",
                    "bilby": bilby_root_path + "GW170817_TF2_with_tukey_fix_result.json"},
              
              "GW170817_NRTidalv2": {"jim": jim_root_path + "GW170817_NRTidalv2/outdir/results_production.npz",
                                     "bilby": bilby_root_path + "GW170817_IMRPhenomD_NRTidalv2_result.json",
                    },
              
              "GW190425_TaylorF2": {"jim": jim_root_path + "GW190425_TaylorF2/outdir_gwosc_data/results_production.npz",
                                    "bilby": bilby_root_path + "GW190425_GWOSC_data_result.json",
                    },
              
              "GW190425_NRTidalv2": {"jim": jim_root_path + "GW190425_NRTidalv2/outdir/results_production.npz",
                                     "bilby": bilby_root_path + "GW190425_NRTv2_GWOSC_data_result.json",
                    },
}

RUN_NAMES = list(paths_dict.keys())
JIM_VAR_NAMES = ['M_c', 'q', 's1_z', 's2_z', 'lambda_1', 'lambda_2', 'd_L', 't_c', 'phase_c', 'cos_iota', 'psi', 'ra', 'sin_dec']
BILBY_VAR_NAMES = ['chirp_mass', 'mass_ratio', 'spin_1z', 'spin_2z', 'lambda_1', 'lambda_2', 'luminosity_distance', 'phase', 'iota', 'psi', 'ra', 'dec']

## ESS methods

In [177]:
def log_prob_to_weights(log_prob):
    log_prob_normalized = log_prob - logsumexp(log_prob)
    weights = np.exp(log_prob_normalized)
    return weights

def kish_ess(weights):
    return (np.sum(weights))**2/np.sum(weights**2)

def compute_ess(log_prob, 
                convert_log_prob_to_weights: bool = False,
                method = "kish"):

    # if log probs are given (in case of jim), convert to weights
    if convert_log_prob_to_weights:
        weights = log_prob_to_weights(log_prob)
    else:
        weights = log_prob # so actually "log prob" is a misnomer...
    
    if method == "rejection_sampling":
        # weights /= np.sum(weights) # to avoid overflow?
        keep = weights > np.random.uniform(0, max(weights), weights.shape)
        ess = np.sum(keep)
        
    elif method == "new_bilby":
        # The new bilby has already rejection sampled --- so N_samples = ESS
        ess = len(weights)
        
    else:
        raise ValueError("Unknown method")
    
    return ess

def compute_arviz_ess(chains, which: str = "smallest"):
    ess_list = []
    chains = np.array(chains).T
    for param_chains in chains:
        ess_list.append(arviz.ess(param_chains))
        
    if which == "smallest":
        ess = np.min(ess_list)
    elif which in ["average", "mean"]:
        ess = np.mean(ess_list)
        
    return ess

Some further utility functions

In [178]:
def my_avg(values):
    """Take the average but floor and int it"""
    
    return int(np.floor(np.mean(list(values))))

def my_get_number(value):
    value = int(np.round(value))
    my_string = str(value)
    first, second = my_string[0], my_string[1]
    
    power = int(np.floor(np.log10(value)))
    
    return_string = r"{}.{} \times 10^{}".format(first, second, power)
    
    return return_string

## Jim

In [179]:
def get_nb_jim_samples(path: str):
    
    identifier = path.split("/")[-3]
    print(f"identifier is: {identifier}")
    
    data = np.load(path)
    chains = data["chains"]
    a, b = np.shape(chains)[0], np.shape(chains)[1]
    
    # Get the chains and the log prob
    data = np.load(path)
    chains = data["chains"]
    chains = np.reshape(chains, (int(a * b), 13))
    log_prob = data["log_prob"]
    log_prob = np.reshape(log_prob, (int(a * b),))
    
    # Check what values are for both methods
    for which_ in ["smallest", "mean"]:
        ess = compute_arviz_ess(chains, which = which_)
        print(f"ESS (which = {which_}) is: {ess}")
    
    ess = compute_arviz_ess(chains, which = "mean")
    
    return a * b, ess

In [180]:
total_nb_samples_jim = {}
ess_jim = {}

for run_name in RUN_NAMES:
    total, ess = get_nb_jim_samples(paths_dict[run_name]["jim"])
    total_nb_samples_jim[run_name] = total
    ess_jim[run_name] = ess
    
print("Total samples for Jim")
print(total_nb_samples_jim)

print("ESS for Jim")
print(ess_jim)

identifier is: GW170817_TaylorF2
ESS (which = smallest) is: 11523.01574505162
ESS (which = mean) is: 18903.66915946
identifier is: GW170817_NRTidalv2
ESS (which = smallest) is: 9566.584275663423
ESS (which = mean) is: 16488.749027760172
identifier is: GW190425_TaylorF2
ESS (which = smallest) is: 51591.248901791354
ESS (which = mean) is: 106992.97257659023
identifier is: GW190425_NRTidalv2
ESS (which = smallest) is: 13444.139475239215
ESS (which = mean) is: 24243.663665017328
Total samples for Jim
{'GW170817_TaylorF2': 220000, 'GW170817_NRTidalv2': 220000, 'GW190425_TaylorF2': 600000, 'GW190425_NRTidalv2': 600000}
ESS for Jim
{'GW170817_TaylorF2': 18903.66915946, 'GW170817_NRTidalv2': 16488.749027760172, 'GW190425_TaylorF2': 106992.97257659023, 'GW190425_NRTidalv2': 24243.663665017328}


## Bilby

In [181]:
def get_nb_bilby_samples(path: str, method = "new_bilby"):
    
    with open(path, "r") as f:
        data = json.load(f)
    posterior = data["posterior"]["content"]
    mc = posterior["chirp_mass"] # just taking the chirp mass samples as an example to get total number of samples
    n = len(mc)
    
    # log_likelihood = np.array(posterior["log_likelihood"])
    # samples = data["samples"]['content']
    nested_samples_weights = data["nested_samples"]['content']["weights"]
    
    ess = int(compute_ess(np.array(nested_samples_weights), convert_log_prob_to_weights=False, method = method))
        
    return n, ess

### pBilby

In [182]:
total_nb_samples_bilby = {}
ess_bilby = {}

for run_name in RUN_NAMES:
    path = paths_dict[run_name]["bilby"]
    total, ess = get_nb_bilby_samples(path, method = "rejection_sampling")
    
    total_nb_samples_bilby[run_name] = total
    ess_bilby[run_name] = ess
    
print("Total samples for Bilby")
print(total_nb_samples_bilby)

print("ESS for Bilby")
print(ess_bilby)

# Get the averages
avg_total_nb_samples_bilby = my_avg(list(total_nb_samples_bilby.values()))
avg_ess_bilby = my_avg(list(ess_bilby.values()))

Total samples for Bilby
{'GW170817_TaylorF2': 44782, 'GW170817_NRTidalv2': 45365, 'GW190425_TaylorF2': 30782, 'GW190425_NRTidalv2': 30277}
ESS for Bilby
{'GW170817_TaylorF2': 6215, 'GW170817_NRTidalv2': 6336, 'GW190425_TaylorF2': 6754, 'GW190425_NRTidalv2': 6918}


### Relative binning-Bilby

In [183]:
def get_nb_rb_bilby_samples(path: str):
    """Different file format -- but these are with the new pBilby version so just return the number of samples as ESS"""
    
    with h5py.File(path, "r") as f:
        posterior = f["posterior"]
        mc = posterior["chirp_mass"][()] # just taking the chirp mass samples as an example
        n = len(mc)
        
        avg_ess = n
    
    return n, avg_ess

In [184]:
rb_paths = ["../RB/gw170817_relbin_TaylorF2_result.hdf5", 
            "../RB/gw170817_relbin_result.hdf5",
            "../RB/gw190425_relbin_TaylorF2_result.hdf5",
            "../RB/gw190425_relbin_result.hdf5"]

total_nb_samples_rb_bilby = {}
ess_rb_bilby = {}

for path, run_name in zip(rb_paths, RUN_NAMES):
    print(run_name)
    total, ess = get_nb_rb_bilby_samples(path)

    total_nb_samples_rb_bilby[run_name] = total
    ess_rb_bilby[run_name] = ess
    
print("Total samples for RB Bilby")
print(total_nb_samples_rb_bilby)

print("ESS for RB Bilby")
print(ess_rb_bilby)

avg_total_nb_samples_rb_bilby = my_avg(list(total_nb_samples_rb_bilby.values()))
avg_ess_rb_bilby = my_avg(list(ess_rb_bilby.values()))

GW170817_TaylorF2
GW170817_NRTidalv2
GW190425_TaylorF2
GW190425_NRTidalv2
Total samples for RB Bilby
{'GW170817_TaylorF2': 5258, 'GW170817_NRTidalv2': 5172, 'GW190425_TaylorF2': 6743, 'GW190425_NRTidalv2': 5172}
ESS for RB Bilby
{'GW170817_TaylorF2': 5258, 'GW170817_NRTidalv2': 5172, 'GW190425_TaylorF2': 6743, 'GW190425_NRTidalv2': 5172}


### ROQ-Bilby

In [185]:
roq_paths = ["/home/thibeau.wouters/TurboPE-BNS/ROQ/gw170817_ROQ_result.hdf5",
             "/home/thibeau.wouters/TurboPE-BNS/ROQ/gw190425_ROQ_result.hdf5"]

total_nb_samples_roq_bilby = {}
ess_roq_bilby = {}

for path, run_name in zip(rb_paths, roq_paths):
    print(run_name)
    total, ess = get_nb_rb_bilby_samples(path)

    total_nb_samples_roq_bilby[run_name] = total
    ess_roq_bilby[run_name] = ess
    
print("Total samples for ROQ Bilby")
print(total_nb_samples_roq_bilby)

print("ESS for ROQ Bilby")
print(ess_roq_bilby)

avg_total_nb_samples_roq_bilby = my_avg(list(total_nb_samples_roq_bilby.values()))
avg_ess_roq_bilby = my_avg(list(ess_roq_bilby.values()))

/home/thibeau.wouters/TurboPE-BNS/ROQ/gw170817_ROQ_result.hdf5
/home/thibeau.wouters/TurboPE-BNS/ROQ/gw190425_ROQ_result.hdf5
Total samples for ROQ Bilby
{'/home/thibeau.wouters/TurboPE-BNS/ROQ/gw170817_ROQ_result.hdf5': 5258, '/home/thibeau.wouters/TurboPE-BNS/ROQ/gw190425_ROQ_result.hdf5': 5172}
ESS for ROQ Bilby
{'/home/thibeau.wouters/TurboPE-BNS/ROQ/gw170817_ROQ_result.hdf5': 5258, '/home/thibeau.wouters/TurboPE-BNS/ROQ/gw190425_ROQ_result.hdf5': 5172}


# Combine into a table

In [186]:
print("Checking the range of number of samples for Jim")
values = list(total_nb_samples_jim.values())
print((np.min(values), np.max(values)))

values = list(ess_jim.values())
print((np.min(values), np.max(values)))

Checking the range of number of samples for Jim
(220000, 600000)
(16488.749027760172, 106992.97257659023)


In [187]:
latex_code = '& Number of samples & Method & Number of effective samples \\\\\n \hline\\hline\n \\textsc{{Jim}} & \\textsc{{Arviz}} ${}$ & ${}$ \\\\ \\hline \n \\textsc{{pBilby}} & Rejection sampling & ${}$ & ${}$ \\\\ \\hline \n \\textsc{{RB-Bilby}} & Rejection sampling & ${}$ & ${}$ \\\\ \\hline \n \\textsc{{ROQ-Bilby}} & Rejection sampling & ${}$ & ${}$ \\\\ \\hline\\hline'\
.format(my_get_number(np.mean(list(total_nb_samples_jim.values()))),
        my_get_number(np.mean(list(ess_jim.values()))),
        my_get_number(avg_total_nb_samples_bilby),
        my_get_number(avg_ess_bilby),
        my_get_number(avg_total_nb_samples_rb_bilby),
        my_get_number(avg_ess_rb_bilby),
        my_get_number(avg_total_nb_samples_roq_bilby),
        my_get_number(avg_ess_roq_bilby)
)

print(latex_code)

# my_get_number(np.min(list(total_nb_samples_jim.values()))) + " -- " + my_get_number(np.max(list(total_nb_samples_jim.values()))),
# my_get_number(np.min(list(ess_jim.values()))) + " -- " + my_get_number(np.max(list(ess_jim.values())))

& Number of samples & Method & Number of effective samples \\
 \hline\hline
 \textsc{Jim} & \textsc{Arviz} $4.1 \times 10^5$ & $4.1 \times 10^4$ \\ \hline 
 \textsc{pBilby} & Rejection sampling & $3.7 \times 10^4$ & $6.5 \times 10^3$ \\ \hline 
 \textsc{RB-Bilby} & Rejection sampling & $5.5 \times 10^3$ & $5.5 \times 10^3$ \\ \hline 
 \textsc{ROQ-Bilby} & Rejection sampling & $5.2 \times 10^3$ & $5.2 \times 10^3$ \\ \hline\hline


## Final table

Only quote ESS to make it more informed.

In [188]:
latex_code = '& Method & ESS \\\\\n \hline\\hline\n \\textsc{{Jim}} & \\textsc{{Arviz}} & ${}$ \\\\ \\hline \n \\textsc{{pBilby}} &  & ${}$ \\\\ \n \\textsc{{RB-Bilby}} & Rejection sampling & ${}$ \\\\ \n \\textsc{{ROQ-Bilby}} & & ${}$ \\\\ \\hline\\hline'\
.format(my_get_number(np.mean(list(ess_jim.values()))),
        my_get_number(avg_ess_bilby),
        my_get_number(avg_ess_rb_bilby),
        my_get_number(avg_ess_roq_bilby)
)

print(latex_code)

& Method & ESS \\
 \hline\hline
 \textsc{Jim} & \textsc{Arviz} & $4.1 \times 10^4$ \\ \hline 
 \textsc{pBilby} &  & $6.5 \times 10^3$ \\ 
 \textsc{RB-Bilby} & Rejection sampling & $5.5 \times 10^3$ \\ 
 \textsc{ROQ-Bilby} & & $5.2 \times 10^3$ \\ \hline\hline
